# 14 – Case Study: Salt Reduction (With Objective Measures)

**Learning Objectives:**
- Understand the UK salt reduction programme as a model public health intervention
- Appreciate the value of objective biomarkers (urinary sodium) in evaluation
- Work through health impact modelling: from intake change to CVD outcomes
- Explore sensitivity analysis and uncertainty in policy evaluation
- Critically assess the evidence for population-level dietary interventions

---

## 1. Background: The UK Salt Reduction Programme

The UK's salt reduction programme is widely regarded as one of the most successful population-level dietary interventions globally.

### Timeline

- **2003**: Food Standards Agency (FSA) launches voluntary salt reduction programme
- **2006**: Voluntary reformulation targets set for food industry
- **2008-2014**: Successive rounds of increasingly stringent targets
- **2011**: Responsibility transferred to Department of Health
- **2014**: Public Health England takes over

### Key Components

1. **Voluntary reformulation**: Industry targets for salt content in processed foods
2. **Labelling**: Traffic light front-of-pack labelling
3. **Public awareness**: Campaigns on salt and health
4. **Monitoring**: Regular surveys of food composition and population intake

### Why Salt Matters

- High sodium intake raises blood pressure
- Elevated blood pressure is a major risk factor for cardiovascular disease (CVD)
- Most dietary sodium comes from processed foods, not table salt
- Population-level reformulation can reduce intake without requiring individual behaviour change

## 2. Setup

In [ ]:
# ============================================================
# Bootstrap cell
# ============================================================

import os
import sys
import pathlib
import subprocess

REPO_URL = "https://github.com/ggkuhnle/phn-epi.git"
REPO_DIR = "phn-epi"

cwd = pathlib.Path.cwd()

if (cwd / "scripts" / "epi_utils.py").is_file():
    repo_root = cwd
elif (cwd.parent / "scripts" / "epi_utils.py").is_file():
    repo_root = cwd.parent
else:
    repo_root = cwd / REPO_DIR
    if not repo_root.is_dir():
        print(f"Cloning repository from {REPO_URL} ...")
        subprocess.run(["git", "clone", REPO_URL, str(repo_root)], check=True)
    else:
        print(f"Using existing repository at {repo_root}")
    os.chdir(repo_root)
    repo_root = pathlib.Path.cwd()

scripts_dir = repo_root / "scripts"
if str(scripts_dir) not in sys.path:
    sys.path.insert(0, str(scripts_dir))

print(f"Repository root: {repo_root}")
print("Bootstrap completed successfully.")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from ipywidgets import interact, FloatSlider, IntSlider, VBox, HBox, Output
import ipywidgets as widgets
from IPython.display import display

from epi_utils import (
    estimate_bp_change_from_sodium,
    estimate_cvd_risk_change,
    calculate_deaths_averted
)

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [10, 6]
np.random.seed(42)

print("Libraries loaded successfully.")

## 3. Measuring Salt Intake: The Gold Standard

### The Problem with Self-Report

Dietary assessment methods (FFQs, 24-hour recalls) have significant limitations for sodium:
- Salt added during cooking/at table is hard to quantify
- Sodium content varies within food categories
- People underreport intake

### 24-Hour Urinary Sodium

The **gold standard** for measuring sodium intake is 24-hour urine collection:
- ~90% of ingested sodium is excreted in urine
- Objective, not reliant on recall
- Can detect true population changes

### Conversion

$$\text{Salt (g)} = \text{Sodium (mmol)} \times 0.0584$$

Or approximately: Salt (g) ≈ Sodium (g) × 2.5

In [ ]:
# UK population salt intake data from 24-hour urinary sodium surveys
# Source: National Diet and Nutrition Survey (NDNS) and dedicated salt surveys

uk_salt_data = pd.DataFrame({
    'year': [2000, 2005, 2008, 2011, 2014, 2019],
    'salt_g_day': [9.5, 9.0, 8.6, 8.1, 8.0, 8.4],  # Illustrative values
    'lower_ci': [9.1, 8.6, 8.2, 7.7, 7.6, 8.0],
    'upper_ci': [9.9, 9.4, 9.0, 8.5, 8.4, 8.8],
    'n': [1500, 1800, 1200, 1000, 800, 600]
})

# Government target
target = 6.0

print("UK Population Salt Intake (24-hour urinary sodium)")
print("=" * 60)
display(uk_salt_data)

In [ ]:
# Visualise the trend
fig, ax = plt.subplots(figsize=(10, 6))

ax.errorbar(uk_salt_data['year'], uk_salt_data['salt_g_day'],
            yerr=[uk_salt_data['salt_g_day'] - uk_salt_data['lower_ci'],
                  uk_salt_data['upper_ci'] - uk_salt_data['salt_g_day']],
            fmt='o-', capsize=5, capthick=2, linewidth=2, markersize=10,
            color='steelblue', label='Measured intake')

ax.axhline(y=target, color='green', linestyle='--', linewidth=2, label=f'Government target ({target}g)')
ax.axhline(y=5.0, color='red', linestyle=':', linewidth=2, label='WHO recommendation (5g)')

# Shade the intervention period
ax.axvspan(2003, 2014, alpha=0.2, color='yellow', label='Active intervention period')

ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Salt intake (g/day)', fontsize=12)
ax.set_title('UK Population Salt Intake: Measured by 24-Hour Urinary Sodium', fontsize=14)
ax.legend(loc='upper right')
ax.set_ylim(4, 11)
ax.set_xlim(1998, 2021)

plt.tight_layout()
plt.show()

# Calculate reduction
reduction = uk_salt_data['salt_g_day'].iloc[0] - uk_salt_data['salt_g_day'].iloc[-2]
pct_reduction = reduction / uk_salt_data['salt_g_day'].iloc[0] * 100

print(f"\nSalt intake reduction (2000-2014): {reduction:.1f} g/day ({pct_reduction:.0f}%)")
print(f"Note: Intake increased slightly between 2014-2019 after programme was deprioritised")

## 4. The Causal Pathway: Salt → Blood Pressure → CVD

### Evidence Base

The causal chain from salt to cardiovascular disease is well established:

1. **Salt → Blood Pressure**: Meta-analyses of RCTs show ~1 mmHg reduction in SBP per 1g/day salt reduction
2. **Blood Pressure → CVD**: Observational studies and trials show clear dose-response

### Key Parameters from Literature

| Parameter | Value | Source |
|-----------|-------|--------|
| SBP change per 1g salt reduction | -1.0 to -1.5 mmHg | He & MacGregor meta-analysis |
| Stroke risk per 2 mmHg SBP | -10% | BPLTTC |
| CHD risk per 2 mmHg SBP | -7% | BPLTTC |

In [ ]:
def estimate_bp_change(salt_reduction_g, effect_per_g=1.2):
    """
    Estimate systolic blood pressure change from salt reduction.
    
    Parameters
    ----------
    salt_reduction_g : float
        Reduction in salt intake (g/day)
    effect_per_g : float
        mmHg SBP reduction per g salt reduction (default 1.2)
    
    Returns
    -------
    float
        Estimated SBP reduction (mmHg)
    """
    return salt_reduction_g * effect_per_g


def estimate_cvd_risk_reduction(sbp_reduction_mmhg, outcome='stroke'):
    """
    Estimate relative risk reduction for CVD outcomes from BP reduction.
    
    Parameters
    ----------
    sbp_reduction_mmhg : float
        Reduction in systolic blood pressure (mmHg)
    outcome : str
        'stroke' or 'chd'
    
    Returns
    -------
    float
        Relative risk reduction (as proportion, e.g., 0.10 = 10% reduction)
    """
    # Risk reduction per 2 mmHg SBP reduction
    rr_per_2mmhg = {'stroke': 0.10, 'chd': 0.07}
    
    # Calculate for actual BP change
    risk_reduction = (sbp_reduction_mmhg / 2) * rr_per_2mmhg[outcome]
    
    return risk_reduction

In [ ]:
# Apply to UK data
salt_reduction = 1.5  # g/day achieved 2000-2014

sbp_reduction = estimate_bp_change(salt_reduction)
stroke_risk_reduction = estimate_cvd_risk_reduction(sbp_reduction, 'stroke')
chd_risk_reduction = estimate_cvd_risk_reduction(sbp_reduction, 'chd')

print("Estimated Health Impact of UK Salt Reduction")
print("=" * 50)
print(f"\nSalt intake reduction: {salt_reduction:.1f} g/day")
print(f"Estimated SBP reduction: {sbp_reduction:.1f} mmHg")
print(f"\nEstimated relative risk reduction:")
print(f"  Stroke: {stroke_risk_reduction:.1%}")
print(f"  CHD: {chd_risk_reduction:.1%}")

## 5. Health Impact Modelling

To estimate deaths or DALYs averted, we need to combine:

1. **Relative risk reduction** (from above)
2. **Baseline event rates** (from vital statistics)
3. **Population size** (from census)

$$\text{Deaths averted} = \text{Population} \times \text{Baseline rate} \times \text{RR reduction}$$

In [ ]:
# UK baseline data (illustrative, approximate)
uk_population_adult = 52_000_000  # Adults 18+

# Annual CVD mortality rates (per 100,000)
baseline_rates = {
    'stroke_mortality': 45,  # per 100,000 per year
    'chd_mortality': 95,     # per 100,000 per year
}

def calculate_deaths_averted_manual(population, baseline_rate_per_100k, 
                                    risk_reduction, years=1):
    """
    Calculate deaths averted from a risk reduction.
    
    Parameters
    ----------
    population : int
        Population size
    baseline_rate_per_100k : float
        Baseline mortality rate per 100,000 per year
    risk_reduction : float
        Relative risk reduction (as proportion)
    years : int
        Number of years
    
    Returns
    -------
    float
        Estimated deaths averted
    """
    baseline_deaths = (population / 100_000) * baseline_rate_per_100k * years
    deaths_averted = baseline_deaths * risk_reduction
    return deaths_averted


# Calculate for UK
stroke_deaths_averted = calculate_deaths_averted_manual(
    uk_population_adult,
    baseline_rates['stroke_mortality'],
    stroke_risk_reduction,
    years=10  # Over 10 years
)

chd_deaths_averted = calculate_deaths_averted_manual(
    uk_population_adult,
    baseline_rates['chd_mortality'],
    chd_risk_reduction,
    years=10
)

total_cvd_deaths_averted = stroke_deaths_averted + chd_deaths_averted

print("Estimated Deaths Averted (10-year period)")
print("=" * 50)
print(f"\nSalt reduction: {salt_reduction:.1f} g/day")
print(f"Population: {uk_population_adult:,}")
print(f"\nStroke deaths averted: {stroke_deaths_averted:,.0f}")
print(f"CHD deaths averted: {chd_deaths_averted:,.0f}")
print(f"\nTotal CVD deaths averted: {total_cvd_deaths_averted:,.0f}")
print(f"Per year: {total_cvd_deaths_averted/10:,.0f}")

## 6. Sensitivity Analysis

Our estimates depend on several uncertain parameters. Sensitivity analysis explores how results change when we vary these assumptions.

In [ ]:
# One-way sensitivity analysis: vary the BP effect per gram of salt

effect_range = np.linspace(0.5, 2.0, 20)  # mmHg per g salt
deaths_by_effect = []

for effect in effect_range:
    sbp = estimate_bp_change(salt_reduction, effect_per_g=effect)
    stroke_rr = estimate_cvd_risk_reduction(sbp, 'stroke')
    chd_rr = estimate_cvd_risk_reduction(sbp, 'chd')
    
    stroke_deaths = calculate_deaths_averted_manual(
        uk_population_adult, baseline_rates['stroke_mortality'], stroke_rr, years=10)
    chd_deaths = calculate_deaths_averted_manual(
        uk_population_adult, baseline_rates['chd_mortality'], chd_rr, years=10)
    
    deaths_by_effect.append(stroke_deaths + chd_deaths)

# Plot
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(effect_range, np.array(deaths_by_effect)/1000, 'b-', linewidth=2)
ax.axvline(x=1.2, color='red', linestyle='--', label='Base case (1.2 mmHg/g)')
ax.fill_between(effect_range, 0, np.array(deaths_by_effect)/1000, alpha=0.3)

ax.set_xlabel('Blood pressure effect (mmHg per g salt reduction)', fontsize=12)
ax.set_ylabel('CVD deaths averted (thousands, 10 years)', fontsize=12)
ax.set_title('Sensitivity Analysis: Effect of BP-Salt Relationship', fontsize=14)
ax.legend()

plt.tight_layout()
plt.show()

print(f"Deaths averted range: {min(deaths_by_effect):,.0f} to {max(deaths_by_effect):,.0f}")

In [ ]:
# Probabilistic sensitivity analysis (Monte Carlo)

n_simulations = 10000

# Parameter distributions
salt_reduction_samples = np.random.normal(1.5, 0.3, n_simulations)  # g/day
effect_per_g_samples = np.random.normal(1.2, 0.3, n_simulations)    # mmHg/g
stroke_rr_per_2mmhg_samples = np.random.normal(0.10, 0.02, n_simulations)
chd_rr_per_2mmhg_samples = np.random.normal(0.07, 0.015, n_simulations)

deaths_averted_samples = []

for i in range(n_simulations):
    sbp = salt_reduction_samples[i] * effect_per_g_samples[i]
    
    stroke_rr = (sbp / 2) * stroke_rr_per_2mmhg_samples[i]
    chd_rr = (sbp / 2) * chd_rr_per_2mmhg_samples[i]
    
    stroke_deaths = (uk_population_adult / 100_000) * baseline_rates['stroke_mortality'] * stroke_rr * 10
    chd_deaths = (uk_population_adult / 100_000) * baseline_rates['chd_mortality'] * chd_rr * 10
    
    deaths_averted_samples.append(stroke_deaths + chd_deaths)

deaths_averted_samples = np.array(deaths_averted_samples)

# Results
mean_deaths = np.mean(deaths_averted_samples)
ci_lower = np.percentile(deaths_averted_samples, 2.5)
ci_upper = np.percentile(deaths_averted_samples, 97.5)

print("Probabilistic Sensitivity Analysis Results")
print("=" * 50)
print(f"\nMean deaths averted (10 years): {mean_deaths:,.0f}")
print(f"95% credible interval: [{ci_lower:,.0f} - {ci_upper:,.0f}]")

In [ ]:
# Visualise distribution
fig, ax = plt.subplots(figsize=(10, 6))

ax.hist(deaths_averted_samples/1000, bins=50, density=True, alpha=0.7, color='steelblue')
ax.axvline(x=mean_deaths/1000, color='red', linestyle='-', linewidth=2, label=f'Mean: {mean_deaths/1000:.0f}k')
ax.axvline(x=ci_lower/1000, color='orange', linestyle='--', linewidth=2, label=f'95% CI: [{ci_lower/1000:.0f}k - {ci_upper/1000:.0f}k]')
ax.axvline(x=ci_upper/1000, color='orange', linestyle='--', linewidth=2)

ax.set_xlabel('CVD deaths averted (thousands, 10 years)', fontsize=12)
ax.set_ylabel('Density', fontsize=12)
ax.set_title('Probabilistic Sensitivity Analysis: Deaths Averted', fontsize=14)
ax.legend()

plt.tight_layout()
plt.show()

## 7. Interactive Health Impact Model

Explore how different assumptions affect the estimated health impact.

In [ ]:
# Interactive model
salt_slider = FloatSlider(value=1.5, min=0.5, max=3.0, step=0.1, description='Salt reduction (g):')
bp_effect_slider = FloatSlider(value=1.2, min=0.5, max=2.0, step=0.1, description='BP effect (mmHg/g):')
years_slider = IntSlider(value=10, min=1, max=20, step=1, description='Time horizon (years):')

output = Output()

def update_model(change=None):
    salt_red = salt_slider.value
    bp_effect = bp_effect_slider.value
    years = years_slider.value
    
    sbp = salt_red * bp_effect
    stroke_rr = (sbp / 2) * 0.10
    chd_rr = (sbp / 2) * 0.07
    
    stroke_deaths = calculate_deaths_averted_manual(
        uk_population_adult, baseline_rates['stroke_mortality'], stroke_rr, years)
    chd_deaths = calculate_deaths_averted_manual(
        uk_population_adult, baseline_rates['chd_mortality'], chd_rr, years)
    total = stroke_deaths + chd_deaths
    
    with output:
        output.clear_output(wait=True)
        
        print(f"\n{'='*50}")
        print(f"HEALTH IMPACT ESTIMATE")
        print(f"{'='*50}")
        print(f"\nInputs:")
        print(f"  Salt reduction: {salt_red:.1f} g/day")
        print(f"  BP effect: {bp_effect:.1f} mmHg per g salt")
        print(f"  Time horizon: {years} years")
        print(f"\nIntermediate calculations:")
        print(f"  SBP reduction: {sbp:.1f} mmHg")
        print(f"  Stroke risk reduction: {stroke_rr:.1%}")
        print(f"  CHD risk reduction: {chd_rr:.1%}")
        print(f"\nEstimated deaths averted:")
        print(f"  Stroke: {stroke_deaths:,.0f}")
        print(f"  CHD: {chd_deaths:,.0f}")
        print(f"  TOTAL: {total:,.0f}")
        print(f"  Per year: {total/years:,.0f}")

for slider in [salt_slider, bp_effect_slider, years_slider]:
    slider.observe(update_model, names='value')

print("Adjust the parameters to see how the health impact estimate changes:")
display(VBox([salt_slider, bp_effect_slider, years_slider, output]))
update_model()

## 8. Strengths of the UK Programme

### Why Is This a Model Intervention?

1. **Objective outcome measurement**: 24-hour urinary sodium provides unbiased evidence of population intake change

2. **Clear causal pathway**: Salt → BP → CVD is well established from multiple study designs

3. **Population-level approach**: Reformulation reduces intake without requiring individual behaviour change

4. **Regular monitoring**: Repeated surveys allow tracking of progress

5. **Industry engagement**: Voluntary targets achieved significant reformulation

### Limitations

1. **No control group**: Cannot prove causality at population level
2. **Other secular trends**: Diet and CVD risk changed for other reasons too
3. **Survey representativeness**: 24-hour urine collection has low response rates
4. **Lag times**: Full health impact takes years to materialise

## 9. Discussion Questions

1. **The measurement advantage**: Why is urinary sodium considered more reliable than dietary assessment? What are its limitations?

2. **Voluntary vs mandatory**: The UK used voluntary industry targets. What are the trade-offs compared to mandatory salt limits (as used in some countries)?

3. **Equity implications**: Is reformulation equitable? Who benefits most from processed food reformulation?

4. **Transferability**: Would this approach work for sugar reduction? Why might sugar be harder than salt?

5. **The 2019 uptick**: Salt intake appeared to increase between 2014-2019 after the programme was deprioritised. What does this suggest about the sustainability of voluntary approaches?

## 10. Exercises

### Exercise 1: Calculate Impact of Achieving the 6g Target

If the UK achieved the 6g/day target (from current ~8g), estimate the additional deaths averted per year.

In [ ]:
# YOUR CODE HERE



### Exercise 2: Sensitivity to Baseline Rate

Repeat the analysis for a country with higher CVD mortality (e.g., Eastern Europe with ~200 per 100,000 for CHD). How does this affect the absolute number of deaths averted?

In [ ]:
# YOUR CODE HERE



---

## Summary

- The UK salt reduction programme achieved a measurable ~15% reduction in population salt intake
- 24-hour urinary sodium provides objective evidence of population intake change
- Health impact modelling combines intake changes, dose-response relationships, and baseline rates
- Sensitivity analysis reveals the uncertainty in estimates (wide credible intervals)
- The programme demonstrates the value of objective biomarkers in evaluating dietary interventions

---

## References

- He FJ, Pombo-Rodrigues S, MacGregor GA. (2014). Salt reduction in England from 2003 to 2011. *BMJ Open*.
- He FJ, Li J, MacGregor GA. (2013). Effect of longer term modest salt reduction on blood pressure. *Cochrane Database*.
- Lewington S et al. (2002). Age-specific relevance of usual blood pressure to vascular mortality. *Lancet*.
- Public Health England. (2020). Salt targets 2024.